# Access to Healthy Assets and Hazards Input Data Creation


# Setup

In [ ]:
# Install required packages for this notebook:
# - bs4: BeautifulSoup for HTML parsing
# - selenium: browser automation (useful for JS-driven pages)
# - earthengine-api: Google Earth Engine Python client (requires authentication)
# Use the Jupyter %pip magic so packages are installed into the notebook environment.
# After installing earthengine-api, run ee.Authenticate() and ee.Initialize() before using GEE.
%pip install bs4 selenium earthengine-api

Note: you may need to restart the kernel to use updated packages.


In [29]:
from pathlib import Path  # For handling file paths in a cross-platform way
import pandas as pd  # For data manipulation and analysis
import geopandas as gpd  # For geospatial data handling
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
import re  # For regular expressions
import requests  # Duplicate import (consider removing)
import time  # For time-related functions
from typing import Optional  # For type hints
import io  # For in-memory I/O operations
import os  # For operating system interfaces
import tempfile  # For temporary file creation
import zipfile  # For ZIP file handling
import urllib.request  # For URL retrieval
import ee  # For Google Earth Engine API
import subprocess  # For running subprocesses

# Boundaries

This creates a consolidated GB Boundary using 2021/22 Census definitions: Lower Layer Super Output Areas (LSOAs) for England and Wales and Data Zones (DZs) for Scotland.

## England and Wales

In [163]:
BASE_URL = (
    "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/"
    "Lower_layer_Super_Output_Areas_December_2021_Boundaries_EW_BGC_V5/FeatureServer/0/query"
)

session = requests.Session()

def fetch_chunk(offset: int, limit: int = 2000) -> gpd.GeoDataFrame:
    params = {
        "where": "1=1",
        "outFields": "LSOA21CD",
        "outSR": "4326",
        "f": "geojson",
        "resultOffset": offset,
        "resultRecordCount": limit,
    }
    r = session.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    return gpd.read_file(io.BytesIO(r.content))

chunks = []
offset = 0
limit = 2000  # typical polygon maxRecordCount

while True:
    g = fetch_chunk(offset, limit)
    if g.empty:
        break
    chunks.append(g)
    # if we got fewer than limit, we've hit the end
    if len(g) < limit:
        break
    offset += limit

gdf = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
gdf = gdf.set_crs(epsg=4326, allow_override=True)
gdf = gdf.rename(columns={"LSOA21CD": "LSOA_DZ_SDZ_21_22"})
gdf = gdf.to_crs(epsg=27700)

# Scotland

In [ ]:
# create a temporary directory and download the zip there
tmp_dir = tempfile.mkdtemp()

# download and import Scottish Data Zone 2022 shapefile into sg_dz_gdf
sg_zip_url = "https://maps.gov.scot/ATOM/shapefiles/SG_DataZoneBdry_2022.zip"
sg_zip_path = os.path.join(tmp_dir, "SG_DataZoneBdry_2022.zip")
urllib.request.urlretrieve(sg_zip_url, sg_zip_path)

# extract
with zipfile.ZipFile(sg_zip_path, 'r') as zip_ref:
    zip_ref.extractall(tmp_dir)

# read
sg_dz_gdf = gpd.read_file(os.path.join(tmp_dir, "SG_DataZone_Bdry_2022.shp"))

sg_dz_gdf = sg_dz_gdf.rename(columns={"dzcode": "LSOA_DZ_SDZ_21_22"})
sg_dz_gdf = sg_dz_gdf[["LSOA_DZ_SDZ_21_22", "geometry"]].copy()

,LSOA_DZ_SDZ_21_22,geometry
0,S01013482,"POLYGON ((383385 800703, 383385.411 800699.959..."
1,S01013483,"POLYGON ((383304.495 801654.205, 383391.947 80..."
2,S01013484,"POLYGON ((383473 801227, 383597 801087, 383598..."
3,S01013485,"POLYGON ((383976.659 801182.579, 383984.102 80..."
4,S01013486,"POLYGON ((384339 801211, 384316.51 801182.159,..."
...,...,...
7387,S01020870,"POLYGON ((307423.889 672579.756, 307424.594 67..."
7388,S01020871,"POLYGON ((308463.398 672524.398, 308451.095 67..."
7389,S01020872,"POLYGON ((308734 672598, 308743 672543, 308905..."
7390,S01020873,"POLYGON ((309949.893 672738.667, 309970.518 67..."


In [175]:
# Combine gdf (England and Wales) and sg_dz_gdf (Scotland) into a single GeoDataFrame
boundaries_all = gpd.GeoDataFrame(
    pd.concat([gdf, sg_dz_gdf], ignore_index=True),
    geometry="geometry",
    crs=gdf.crs
)

# Export to GeoParquet
out_dir = Path("data") / "boundary"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "LSOA_DZ_SDZ_21_22.parquet"
boundaries_all.to_parquet(out_path, index=False)
print("Saved boundaries geoparquet to", out_path)

Saved boundaries geoparquet to data/boundary/LSOA_DZ_SDZ_21_22.parquet


# Import Postcodes

Imports the ONSPD (Office for National Statistics Postcode Directory) data, and limits these to those within GB.

In [30]:
# Load ONS Postcode Directory
onspd_path = Path("data") / "raw_data"/ "ONSPD_NOV_2025_UK.csv"

# The ONSPD file is large, so we only load the columns we need
# Key columns: pcds (postcode), oseast1m (easting), osnrth1m (northing)
postcodes_raw = pd.read_csv(
    onspd_path,
    usecols=["pcd7", "lat", "long","lsoa21cd","doterm"],
    dtype={"pcd7": str, "east1m": float, "north1m": float}
)

# remove spaces from pcd7 and keep only rows where doterm is empty
postcodes_raw["pcd7"] = postcodes_raw["pcd7"].str.replace(" ", "", regex=False)
mask = (
    (postcodes_raw["doterm"].isna() | (postcodes_raw["doterm"].astype(str).str.strip() == ""))
    & postcodes_raw["lsoa21cd"].notna()
)
postcodes_raw = postcodes_raw.loc[mask].copy()

# remove rows with invalid LSOA codes
invalid_lsoa = {"L99999999", "M99999999"}
postcodes_raw = postcodes_raw[~postcodes_raw["lsoa21cd"].isin(invalid_lsoa)].copy()

postcodes_raw = postcodes_raw.drop(columns=["doterm"], errors="ignore")
print(f"Loaded {len(postcodes_raw):,} postcodes")
postcodes_raw

# remove rows where lsoa21cd starts with 'N'
postcodes_raw = postcodes_raw[~postcodes_raw["lsoa21cd"].astype(str).str.startswith("N", na=False)].copy()

# convert postcodes_raw to a GeoDataFrame
postcodes_geo = postcodes_raw.dropna(subset=["lat", "long"]).copy()
postcodes_gdf = gpd.GeoDataFrame(
    postcodes_geo,
    geometry=gpd.points_from_xy(postcodes_geo["long"], postcodes_geo["lat"]),
    crs="EPSG:4326",
)
# Report how many rows were converted
print(f"Converted {len(postcodes_gdf):,} rows to GeoDataFrame (dropped {len(postcodes_raw) - len(postcodes_gdf):,} without coords)")

# Save postcodes_gdf as GeoParquet
postcode_out_dir = Path("data") / "postcodes"
postcode_out_dir.mkdir(parents=True, exist_ok=True)
out_path = postcode_out_dir / "postcodes.parquet"
postcodes_gdf.to_parquet(out_path, index=False)

Loaded 1,794,259 postcodes
Converted 1,745,013 rows to GeoDataFrame (dropped 0 without coords)


# Import Retail Data

Creates POI data for retail / leisure locations from the Local Data Company (LDC) dataset.

In [144]:
# Load LDC Data
LDC_path = Path("data") / "raw_data" / "LDC_Historical_snapshot_2015-01-01-2025-09-30.csv"

LDC_raw = pd.read_csv(
    LDC_path,
    usecols=["Name", "Latitude", "Longitude","Status","Category","Subcategory","Postcode"],
    dtype={"Name": str, "Latitude": float, "Longitude": float,"Status": str,"Category": str,"Subcategory": str},
)

LDC_raw = LDC_raw[LDC_raw["Status"].str.strip().str.lower() == "live"].copy()

# Load category to subcategory mapping
category_subcategory = pd.read_csv(Path("data") / "raw_data" /"category_subcategory.csv")

# Append category_subcategory onto LDC_raw using Subcategory
LDC_raw = LDC_raw.merge(
    category_subcategory[["Subcategory","Fast_Food","Pub_Bar","Gambling","Tobacco","Leisure"]],
    on="Subcategory",
    how="left",
)

print(f"Loaded {len(LDC_raw):,} stores")

# convert LDC_raw to a GeoDataFrame

LDC_geo = LDC_raw.dropna(subset=["Latitude", "Longitude"]).copy()
LDC_gdf = gpd.GeoDataFrame(
    LDC_geo,
    geometry=gpd.points_from_xy(LDC_geo["Longitude"], LDC_geo["Latitude"]),
    crs="EPSG:4326",
)

print(f"Converted {len(LDC_gdf):,} rows to GeoDataFrame (dropped {len(LDC_raw) - len(LDC_gdf):,} without coords)")

Loaded 874,810 stores
Converted 874,314 rows to GeoDataFrame (dropped 496 without coords)


### Create Retail / Leisure POI Parquet Files

In [145]:
# Keep geometry so outputs are GeoParquet
cols_to_keep = ["Name", "Latitude", "Longitude", "Postcode","geometry"]

def _subset_geo(flag_col: str) -> gpd.GeoDataFrame:
    m = LDC_gdf[flag_col].fillna(0).astype(int).eq(1)
    out = LDC_gdf.loc[m, cols_to_keep].copy()
    return gpd.GeoDataFrame(out, geometry="geometry", crs=LDC_gdf.crs)

df_fast_food = _subset_geo("Fast_Food")
df_pub_bar = _subset_geo("Pub_Bar")
df_gambling = _subset_geo("Gambling")
df_tobacco = _subset_geo("Tobacco")
df_leisure = _subset_geo("Leisure")

# write outputs to data/retail (GeoParquet)
out_dir = Path("data") / "retail"
out_dir.mkdir(parents=True, exist_ok=True)

df_fast_food.to_parquet(out_dir / "fast_food.parquet", index=False)
df_pub_bar.to_parquet(out_dir / "pub_bar.parquet", index=False)
df_gambling.to_parquet(out_dir / "gambling.parquet", index=False)
df_tobacco.to_parquet(out_dir / "tobacco.parquet", index=False)
df_leisure.to_parquet(out_dir / "leisure.parquet", index=False)

print("Saved GeoParquet files to", out_dir)

Saved GeoParquet files to data/retail


# Health Services

## Hospitals

## Scotland

The hopital data for Scotland is not available from a direct download link (https://data.spatialhub.scot/dataset/nhs_hospitals-is/) and requires a log on to be created. The data are OGL, and have been placed in ./data/raw_data/NHS_Hospitals_-_Scotland.csv

In [219]:
# Load CSV
scot_path = Path("data") / "raw_data" / "NHS_Hospitals_-_Scotland.csv"

# Load only the required columns
hospitals_scotland = pd.read_csv(
    scot_path,
    usecols=["sitename", "postcode", "x", "y"],
    low_memory=False,
    dtype={"sitename": str, "postcode": str, "x": float, "y": float},
)
# Clean postcodes by removing spaces
hospitals_scotland['postcode'] = hospitals_scotland['postcode'].str.replace(' ', '', regex=False)


hospitals_scotland = hospitals_scotland.groupby(["postcode", "x", "y"], as_index=False).agg({
    "sitename": lambda sitename: " | ".join(sitename.astype(str).str.strip().unique())
})

# Create GeoDataFrame (assuming x,y are OSGB36 eastings/northings)
hospitals_scotland = gpd.GeoDataFrame(
    hospitals_scotland,
    geometry=gpd.points_from_xy(hospitals_scotland["x"], hospitals_scotland["y"]),
    crs="EPSG:27700",
)

# Convert to WGS84 and add lat/lon, rename columns to match others
hospitals_scotland = hospitals_scotland.to_crs("EPSG:4326") # WGS84
hospitals_scotland["Latitude"] = hospitals_scotland.geometry.y # lat
hospitals_scotland["Longitude"] = hospitals_scotland.geometry.x # lon
hospitals_scotland = hospitals_scotland.rename(columns={"sitename": "Name", "postcode": "Postcode"}) # rename columns
hospitals_scotland = hospitals_scotland[["Name", "Latitude", "Longitude", "Postcode", "geometry"]].copy() # keep only relevant columns

### England and Wales
The hospital data for England and Wales is available as a direct download link from: https://digital.nhs.uk/services/organisation-data-service/data-search-and-export/csv-downloads/other-nhs-organisations

In [220]:
# Import GP practice data for England and Wales
hospitals_england_wales_url = "https://www.odsdatasearchandexport.nhs.uk/api/getReport?report=ets"
hospitals_england_wales = pd.read_csv(hospitals_england_wales_url, header=None, low_memory=False, usecols=[1,9,12])

hospitals_england_wales.to_csv("./data/raw_data/hospitals_england_wales.csv", index=False) # Save a copy of the raw data

# Keep only rows where column 12 (Status) is empty (indicating active hospitals)
hospitals_england_wales = hospitals_england_wales[hospitals_england_wales[12].isna()].copy()

hospitals_england_wales.columns = ['Name', 'Postcode', 'Status']

# Filter out rows where Postcode starts with 'JE' or 'IM' (Jersey / Isle of Man / Guernsey / British Forces)
hospitals_england_wales = hospitals_england_wales[~hospitals_england_wales['Postcode'].str.startswith(('JE', 'IM','GY','BF'))]

# Clean postcodes by removing spaces and uppercasing
hospitals_england_wales["Postcode"] = hospitals_england_wales["Postcode"].str.replace(" ", "", regex=False).str.upper()
hospitals_england_wales = hospitals_england_wales.drop(columns=["Status"], errors="ignore")

hospitals_england_wales = hospitals_england_wales.groupby("Postcode", as_index=False).agg({
    "Name": lambda names: " | ".join(names.astype(str).str.strip().unique())
})

# Merge with postcodes on Postcode and pcd7
hospitals_england_wales = hospitals_england_wales.merge(
    postcodes[["pcd7", "lat", "long"]],
    left_on="Postcode",
    right_on="pcd7",
    how="left"
)

# Rename columns to match the standard format
hospitals_england_wales = hospitals_england_wales.rename(columns={
    "lat": "Latitude",
    "long": "Longitude"
})

# Filter out rows where Postcode starts with 'JE' or 'IM' or 'GY' (Jersey / Isle of Man / Guernsey)
hospitals_england_wales = hospitals_england_wales[~hospitals_england_wales['Postcode'].str.startswith(('JE', 'IM','GY'))]

# Select final columns
hospitals_england_wales = hospitals_england_wales[["Name", "Latitude", "Longitude", "Postcode"]].copy()

# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = hospitals_england_wales[hospitals_england_wales['Latitude'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = hospitals_england_wales.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            Latitude, Longitude = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                Latitude = data['result']['Latitude']
                Longitude = data['result']['Longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                Latitude = data['terminated']['latitude']
                Longitude = data['terminated']['longitude']
            
            if Latitude is not None and Longitude is not None:
                hospitals_england_wales.at[idx, 'Latitude'] = Latitude
                hospitals_england_wales.at[idx, 'Longitude'] = Longitude
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")


# Convert to GeoDataFrame
# Remove rows where Latitude or Longitude are NaN
hospitals_england_wales = hospitals_england_wales.dropna(subset=['Latitude', 'Longitude'])

# Create GeoDataFrame 
hospitals_england_wales = gpd.GeoDataFrame(
    hospitals_england_wales,
    geometry=gpd.points_from_xy(hospitals_england_wales['Longitude'], hospitals_england_wales['Latitude']),
    crs='EPSG:4326')

Error fetching data for postcode BT126BA: 'Latitude'
Error fetching data for postcode BT126BE: 'Latitude'
Error fetching data for postcode BT97AB: 'Latitude'


The folloiwing code creates a consolidated hospital POI dataset for GB.

In [221]:
# Combine Scotland and England/Wales hospitals and export to GeoParquet

print(f"Scotland hospitals: {len(hospitals_scotland)}")
print(f"England and Wales hospitals: {len(hospitals_england_wales)}")

hospitals_all = gpd.GeoDataFrame(
    pd.concat([hospitals_england_wales, hospitals_scotland], ignore_index=True),
    geometry="geometry",
    crs=hospitals_england_wales.crs
)

health_dir = Path("data") / "health"
health_dir.mkdir(parents=True, exist_ok=True)
hospitals_out_path = health_dir / "hospital.parquet"
hospitals_all.to_parquet(hospitals_out_path, index=False)
print("Saved hospitals geoparquet to", hospitals_out_path)

Scotland hospitals: 295
England and Wales hospitals: 17810
Saved hospitals geoparquet to data/health/hospital.parquet


## GP Practices
The following code creates a consolidated GP Practice POI dataset for GB. There is some noise in the data, with some practices having repeated entries. These were all treated as single locations. Furthermore, some practices have postcodes which are no longer current. These records were all retained but geocoded using postcodes.io, as the main postcode file imported earlier would not have these postcodes.
### Scotland
The GP practice data for Scotland is available from a direct download link (https://www.opendata.nhs.scot/dataset/gp-practice-contact-details-and-list-sizes).

In [222]:
# Import GP practice data for Scotland (Oct 2025)
gp_scotland_url = "https://www.opendata.nhs.scot/dataset/f23655c3-6e23-4103-a511-a80d998adb90/resource/47557411-7eda-4278-9d6d-d26ed2ceab5a/download/practice_contact_details_20251001_opendata.csv"
gp_scotland = pd.read_csv(gp_scotland_url, low_memory=False)

gp_scotland.to_csv("./data/raw_data/gp_scotland_20251001_raw.csv", index=False) # Save a copy of the raw data

# Keep only relevant columns
gp_scotland = gp_scotland[["GPPracticeName", "Postcode"]]
gp_scotland["Postcode"] = gp_scotland["Postcode"].str.replace(" ", "", regex=False)

# Aggregate by Postcode to combine multiple practices in the same postcode
gp_scotland = gp_scotland.groupby("Postcode", as_index=False).agg({
    "GPPracticeName": lambda GPPracticeName: " | ".join(GPPracticeName.astype(str).str.strip().unique())
})

# Join postcodes using a match between Postcode and pcd7
gp_scotland = gp_scotland.merge(
    postcodes[["pcd7", "lat", "long"]],
    left_on="Postcode",
    right_on="pcd7",
    how="left"
)

# Rename and select final columns
gp_scotland = gp_scotland.rename(columns={
    "GPPracticeName": "Name",
    "lat": "Latitude",
    "long": "Longitude"
})

# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = gp_scotland[gp_scotland['Latitude'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = gp_scotland.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            Latitude, Longitude = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                Latitude = data['result']['Latitude']
                Longitude = data['result']['Longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                Latitude = data['terminated']['latitude']
                Longitude = data['terminated']['longitude']
            
            if Latitude is not None and Longitude is not None:
                gp_scotland.at[idx, 'Latitude'] = Latitude
                gp_scotland.at[idx, 'Longitude'] = Longitude
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")

# Select final columns
gp_scotland = gp_scotland[["Name", "Latitude", "Longitude", "Postcode"]]

gp_scotland = gpd.GeoDataFrame(
    gp_scotland,
    geometry=gpd.points_from_xy(gp_scotland["Longitude"], gp_scotland["Latitude"]),
    crs="EPSG:4326",
)


### England and Wales
The GP practice data for England and Wales are available from a direct download link [(https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/2023-24-01-august-2023).](https://digital.nhs.uk/services/organisation-data-service/data-search-and-export/csv-downloads/gp-and-gp-practice-related-data)

In [226]:
# Import GP practice data for England and Wales
gp_england_wales_url = "https://www.odsdatasearchandexport.nhs.uk/api/getReport?report=epraccur"
gp_england_wales = pd.read_csv(gp_england_wales_url, header=None, low_memory=False, usecols=[1,9,12])

gp_england_wales.to_csv("./data/raw_data/gp_england_wales_raw.csv", index=False) # Save a copy of the raw data

gp_england_wales = gp_england_wales[gp_england_wales[12] == 'ACTIVE']

gp_england_wales.columns = ['Name', 'Postcode', 'Status']


# Filter out rows where Postcode starts with 'JE' or 'IM' or 'GY' (Jersey / Isle of Man / Guernsey) and British Forces
gp_england_wales = gp_england_wales[~gp_england_wales['Postcode'].str.startswith(('JE', 'IM','GY','BF'))]

gp_england_wales = gp_england_wales.drop(columns=["Status"], errors="ignore")

# Clean postcodes by removing spaces and uppercasing
gp_england_wales["Postcode"] = gp_england_wales["Postcode"].str.replace(" ", "", regex=False).str.upper()

gp_england_wales = gp_england_wales.groupby("Postcode", as_index=False).agg({
    "Name": lambda names: " | ".join(names.astype(str).str.strip().unique())
})


# Merge with postcodes on Postcode and pcd7
gp_england_wales = gp_england_wales.merge(
    postcodes[["pcd7", "lat", "long", "geometry"]],
    left_on="Postcode",
    right_on="pcd7",
    how="left"
)

# Rename columns to match the standard format
gp_england_wales = gp_england_wales.rename(columns={
    "lat": "Latitude",
    "long": "Longitude"
})

# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = gp_england_wales[gp_england_wales['Latitude'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = gp_england_wales.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            Latitude, Longitude = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                Latitude = data['result']['Latitude']
                Longitude = data['result']['Longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                Latitude = data['terminated']['latitude']
                Longitude = data['terminated']['longitude']
            
            if Latitude is not None and Longitude is not None:
                gp_england_wales.at[idx, 'Latitude'] = Latitude
                gp_england_wales.at[idx, 'Longitude'] = Longitude
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")


# Select final columns
gp_england_wales = gp_england_wales[["Name", "Latitude", "Longitude", "Postcode"]].copy()

gp_england_wales = gpd.GeoDataFrame(
    gp_england_wales,
    geometry=gpd.points_from_xy(gp_england_wales["Longitude"], gp_england_wales["Latitude"]),
    crs="EPSG:4326",
)


In [227]:
# Combine GP practices from England, Wales, and Scotland
print(f"Scotland GPs: {len(gp_scotland)}")
print(f"England and Wales GPs: {len(gp_england_wales)}")

gp_practices = gpd.GeoDataFrame(
    pd.concat([gp_england_wales, gp_scotland], ignore_index=True),
    geometry="geometry",
    crs=postcodes.crs
)   

health_dir = Path("data") / "health"
health_dir.mkdir(parents=True, exist_ok=True)

# Export combined GP practices as GeoParquet
gp_out_path = health_dir / "GP.parquet"
gp_practices.to_parquet(gp_out_path, index=False)
print("Saved GP practices geoparquet to", gp_out_path)


Scotland GPs: 706
England and Wales GPs: 8423
Saved GP practices geoparquet to data/health/GP.parquet


## Pharmacies
### Scotland
The pharmacy data for Scotland is available from a direct download - https://www.opendata.nhs.scot/dataset/dispenser-location-contact-details/

In [232]:
# Import pharmacy data for Scotland
pharmacy_scotland_url = "https://www.opendata.nhs.scot/dataset/a30fde16-1226-49b3-b13d-eb90e39c2058/resource/fba47ad2-9082-4e4a-a70d-16215922c1f7/download/dispenser_contactdetails_jul-sep_2025-26.csv"
pharmacy_scotland = pd.read_csv(pharmacy_scotland_url, low_memory=False)

pharmacy_scotland.to_csv("./data/raw_data/pharmacy_scotland_20251001_raw.csv", index=False) # Save a copy of the raw data

# Select relevant columns (assuming 'DispenserName' and 'Postcode' based on typical structure)
pharmacy_scotland = pharmacy_scotland[["DispLocationName", "DispLocationPostcode"]]

# Clean postcodes by removing spaces and uppercasing
pharmacy_scotland["DispLocationPostcode"] = pharmacy_scotland["DispLocationPostcode"].str.replace(" ", "", regex=False).str.upper()

# Group by postcode to aggregate names
pharmacy_scotland = pharmacy_scotland.groupby("DispLocationPostcode", as_index=False).agg({
    "DispLocationName": lambda DispLocationName: " | ".join(DispLocationName.astype(str).str.strip().unique())
})

# Merge with postcodes using Postcode and pcd7
pharmacy_scotland = pharmacy_scotland.merge(
    postcodes[["pcd7", "lat", "long", "geometry"]],
    left_on="DispLocationPostcode",
    right_on="pcd7",
    how="left"
)

# Rename columns to match standard format
pharmacy_scotland = pharmacy_scotland.rename(columns={
    "DispLocationName": "Name",
    "DispLocationPostcode": "Postcode",
    "lat": "Latitude",
    "long": "Longitude"
})

# Select final columns
pharmacy_scotland = pharmacy_scotland[["Name", "Latitude", "Longitude", "Postcode"]].copy()


# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = pharmacy_scotland[pharmacy_scotland['Latitude'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = pharmacy_scotland.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            Latitude, Longitude = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                Latitude = data['result']['Latitude']
                Longitude = data['result']['Longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                Latitude = data['terminated']['latitude']
                Longitude = data['terminated']['longitude']
            
            if Latitude is not None and Longitude is not None:
                pharmacy_scotland.at[idx, 'Latitude'] = Latitude
                pharmacy_scotland.at[idx, 'Longitude'] = Longitude
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")


# Convert to GeoDataFrame
pharmacy_scotland = gpd.GeoDataFrame(
    pharmacy_scotland,
    geometry=gpd.points_from_xy(pharmacy_scotland["Longitude"], pharmacy_scotland["Latitude"]),
    crs="EPSG:4326",
)

### England and Wales
The GP practice data for England and Wales is available from a direct download link (https://digital.nhs.uk/services/organisation-data-service/data-search-and-export/csv-downloads/gp-and-gp-practice-related-data).

In [236]:
# Import pharmacy data for England and Wales
pharmacy_england_wales_url = "https://www.odsdatasearchandexport.nhs.uk/api/getReport?report=edispensary"
pharmacy_england_wales = pd.read_csv(pharmacy_england_wales_url, header=None, low_memory=False, usecols=[1,9,12])
pharmacy_england_wales.to_csv("./data/raw_data/pharmacy_england_wales_raw.csv", index=False) # Save a copy of the raw data

pharmacy_england_wales = pharmacy_england_wales[pharmacy_england_wales[12] == 'ACTIVE']

pharmacy_england_wales.columns = ['Name', 'Postcode', 'Status']

# Filter out rows where Postcode starts with 'JE' or 'IM' or 'GY' (Jersey / Isle of Man / Guernsey) and British Forces
pharmacy_england_wales = pharmacy_england_wales[~pharmacy_england_wales['Postcode'].str.startswith(('JE', 'IM','GY','BF'))]

# Clean postcodes by removing spaces and uppercasing
pharmacy_england_wales["Postcode"] = pharmacy_england_wales["Postcode"].str.replace(" ", "", regex=False).str.upper()

pharmacy_england_wales = pharmacy_england_wales.drop(columns=["Status"], errors="ignore")

pharmacy_england_wales = pharmacy_england_wales.groupby("Postcode", as_index=False).agg({
    "Name": lambda names: " | ".join(names.astype(str).str.strip().unique())
})

# Merge with postcodes using Postcode and pcd7
pharmacy_england_wales = pharmacy_england_wales.merge(
    postcodes[["pcd7", "lat", "long", "geometry"]],
    left_on="Postcode",
    right_on="pcd7",
    how="left"
)

# Rename columns to match standard format
pharmacy_england_wales = pharmacy_england_wales.rename(columns={
    "lat": "Latitude",
    "long": "Longitude"
})

# Select final columns
pharmacy_england_wales = pharmacy_england_wales[["Name", "Latitude", "Longitude", "Postcode"]].copy()


# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = pharmacy_england_wales[pharmacy_england_wales['Latitude'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = pharmacy_england_wales.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            Latitude, Longitude = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                Latitude = data['result']['Latitude']
                Longitude = data['result']['Longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                Latitude = data['terminated']['latitude']
                Longitude = data['terminated']['longitude']
            
            if Latitude is not None and Longitude is not None:
                pharmacy_england_wales.at[idx, 'Latitude'] = Latitude
                pharmacy_england_wales.at[idx, 'Longitude'] = Longitude
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")



# Convert to GeoDataFrame
pharmacy_england_wales = gpd.GeoDataFrame(
    pharmacy_england_wales,
    geometry=gpd.points_from_xy(pharmacy_england_wales["Longitude"], pharmacy_england_wales["Latitude"]),
    crs="EPSG:4326",
)

In [237]:
# Combine Pharmacy locations from England, Wales, and Scotland

print(f"Scotland pharmacies: {len(pharmacy_scotland)}")
print(f"England and Wales pharmacies: {len(pharmacy_england_wales)}")

pharmacies_all = gpd.GeoDataFrame(
    pd.concat([pharmacy_england_wales, pharmacy_scotland], ignore_index=True),
    geometry="geometry",
    crs=postcodes.crs
)

health_dir = Path("data") / "health"
health_dir.mkdir(parents=True, exist_ok=True)

# Export combined pharmacies as GeoParquet
pharmacy_out_path = health_dir / "pharmacy.parquet"
pharmacies_all.to_parquet(pharmacy_out_path, index=False)
print("Saved pharmacies geoparquet to", pharmacy_out_path)

Scotland pharmacies: 1297
England and Wales pharmacies: 10899
Saved pharmacies geoparquet to data/health/pharmacy.parquet


## Dentists
### Scotland
The dentist data for Scotland is available from a direct download link (https://www.opendata.nhs.scot/dataset/dental-practices-and-patient-registrations)

In [238]:
# Import dentist data for Scotland
dentists_scotland_url = "https://www.opendata.nhs.scot/dataset/2f218ba7-6695-4b22-867d-41383ae36de7/resource/a4eed9fe-bc2a-4a67-a2cd-ce0c765269b8/download/nhs-dental-practices-and-nhs-dental-registrations-as-at-30-jun-2025.csv"
dentists_scotland = pd.read_csv(dentists_scotland_url, low_memory=False)
dentists_scotland.to_csv("./data/raw_data/dentists_scotland_20251001_raw.csv", index=False) # Save a copy of the raw data

# Select relevant columns (assuming 'PracticeName' and 'Postcode' based on typical structure)
dentists_scotland = dentists_scotland[["address1", "pc7"]]

# Clean postcodes by removing spaces and uppercasing
dentists_scotland["pc7"] = dentists_scotland["pc7"].str.replace(" ", "", regex=False).str.upper()

dentists_scotland = dentists_scotland[dentists_scotland['pc7'] != 'PA154LY'] # this had closed

# Group by postcode to aggregate names
dentists_scotland = dentists_scotland.groupby("pc7", as_index=False).agg({
    "address1": lambda address1: " | ".join(address1.astype(str).str.strip().unique())
})

# Merge with postcodes using pc7 and pcd7
dentists_scotland = dentists_scotland.merge(
    postcodes[["pcd7", "lat", "long", "geometry"]],
    left_on="pc7",
    right_on="pcd7",
    how="left"
)

# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = dentists_scotland[dentists_scotland['lat'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = dentists_scotland.at[idx, 'pc7']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            lat, long = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                lat = data['result']['latitude']
                long = data['result']['longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                lat = data['terminated']['latitude']
                long = data['terminated']['longitude']
            
            if lat is not None and long is not None:
                dentists_scotland.at[idx, 'lat'] = lat
                dentists_scotland.at[idx, 'long'] = long
                dentists_scotland.at[idx, 'geometry'] = gpd.points_from_xy([long], [lat], crs="EPSG:4326")[0]
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")

# Rename columns to match standard format
dentists_scotland = dentists_scotland.rename(columns={
    "address1": "Name",
    "pc7": "Postcode",
    "lat": "Latitude",
    "long": "Longitude"
})

# Select final columns
dentists_scotland = dentists_scotland[["Name", "Latitude", "Longitude", "Postcode", "geometry"]].copy()

# Convert to GeoDataFrame
dentists_scotland = gpd.GeoDataFrame(dentists_scotland, geometry="geometry", crs=postcodes.crs)

### England and Wales
The GP practice data for England and Wales is available from a direct download link (https://digital.nhs.uk/services/organisation-data-service/data-search-and-export/csv-downloads/miscellaneous).

In [240]:
# Import dentist data for England and Wales
dentists_england_wales_url = "https://www.odsdatasearchandexport.nhs.uk/api/getReport?report=egdpprac"

dentists_england_wales = pd.read_csv(dentists_england_wales_url, header=None, low_memory=False)
dentists_england_wales.to_csv("./data/raw_data/dentists_england_wales_raw.csv", index=False) # Save a copy of the raw data

#dentists_england_wales = pd.read_csv(dentists_england_wales_url, header=None, low_memory=False, usecols=[1,9,12])
dentists_england_wales = dentists_england_wales[dentists_england_wales[12] == 'ACTIVE']
dentists_england_wales = dentists_england_wales[[1, 9,4,5,7]]
dentists_england_wales.columns = ['Name', 'Postcode', 'Address1', 'Address2', 'Address3']

# Filter out rows where Postcode starts with 'JE' or 'IM' or 'GY' (Jersey / Isle of Man / Guernsey) and British Forces
dentists_england_wales = dentists_england_wales[~dentists_england_wales['Postcode'].str.startswith(('JE', 'IM','GY','BF'))]

# Clean postcodes by removing spaces and uppercasing
dentists_england_wales["Postcode"] = dentists_england_wales["Postcode"].str.replace(" ", "", regex=False).str.upper()

# Merge with postcodes using Postcode and pcd7
dentists_england_wales = dentists_england_wales.merge(
    postcodes[["pcd7", "lat", "long", "geometry"]],
    left_on="Postcode",
    right_on="pcd7",
    how="left"
)

# For any missing lat/long, attempt to fetch from postcodes.io (these are all terminated postcodes which arent in the lookup)
missing = dentists_england_wales[dentists_england_wales['lat'].isna()].copy()
if not missing.empty:
    for idx in missing.index:
        postcode = dentists_england_wales.at[idx, 'Postcode']
        try:
            response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
            data = response.json()
            
            lat, long = None, None
            
            # Active postcode
            if data.get('status') == 200 and 'result' in data:
                lat = data['result']['latitude']
                long = data['result']['longitude']
            # Terminated postcode - still has coordinates
            elif 'terminated' in data:
                lat = data['terminated']['latitude']
                long = data['terminated']['longitude']
            
            if lat is not None and long is not None:
                dentists_england_wales.at[idx, 'lat'] = lat
                dentists_england_wales.at[idx, 'long'] = long
                dentists_england_wales.at[idx, 'geometry'] = gpd.points_from_xy([long], [lat], crs="EPSG:4326")[0]
                
        except Exception as e:
            print(f"Error fetching data for postcode {postcode}: {e}")


# Rename columns to match standard format
dentists_england_wales = dentists_england_wales.rename(columns={
    "lat": "Latitude",
    "long": "Longitude"
})

# Select final columns
dentists_england_wales = dentists_england_wales[["Name", "Latitude", "Longitude", "Postcode", "geometry"]].copy()


dentists_england_wales = dentists_england_wales.groupby(["Postcode", "Latitude", "Longitude","geometry"], as_index=False).agg({
    "Name": lambda Name: " | ".join(Name.astype(str).str.strip().unique())
})


# Convert to GeoDataFrame
dentists_england_wales = gpd.GeoDataFrame(dentists_england_wales, geometry="geometry", crs=postcodes.crs)

In [241]:
# Combine Dentist locations from England, Wales, and Scotland
print(f"Scotland dentists: {len(dentists_scotland)}")
print(f"England and Wales dentists: {len(dentists_england_wales)}")

dentist_all = gpd.GeoDataFrame(
    pd.concat([dentists_england_wales, dentists_scotland], ignore_index=True),
    geometry="geometry",
    crs=postcodes.crs
)

health_dir = Path("data") / "health"
health_dir.mkdir(parents=True, exist_ok=True)

# Export combined dentists as GeoParquet
dentist_out_path = health_dir / "dentist.parquet"
dentist_all.to_parquet(dentist_out_path, index=False)
print("Saved dentists geoparquet to", dentist_out_path)

Scotland dentists: 940
England and Wales dentists: 9106
Saved dentists geoparquet to data/health/dentist.parquet


# Air Quality

Gridded air quality data are all sourced directly from - https://uk-air.defra.gov.uk/data/pcm-data

In [165]:
PM10 = pd.read_csv('https://uk-air.defra.gov.uk/datastore/pcm/mappm102024g.csv', skiprows=5, na_values=['MISSING']) # one values is 'MISSING'
NO2 = pd.read_csv('https://uk-air.defra.gov.uk/datastore/pcm/mapnox2024.csv', skiprows=5)
SO2 = pd.read_csv('https://uk-air.defra.gov.uk/datastore/pcm/mapso22024.csv', skiprows=5)

out_dir = Path("data") / "raw_data"
out_dir.mkdir(parents=True, exist_ok=True)

PM10.to_csv(out_dir / "PM10_2024.csv", index=False) # Save PM10 data
NO2.to_csv(out_dir / "NO2_2024.csv", index=False) # Save NO2 data
SO2.to_csv(out_dir / "SO2_2024.csv", index=False) # Save SO2 data

PM10_gdf = gpd.GeoDataFrame(PM10, geometry=gpd.points_from_xy(PM10['x'], PM10['y']), crs='EPSG:27700')
NO2_gdf = gpd.GeoDataFrame(NO2, geometry=gpd.points_from_xy(NO2['x'], NO2['y']), crs='EPSG:27700')
SO2_gdf = gpd.GeoDataFrame(SO2, geometry=gpd.points_from_xy(SO2['x'], SO2['y']), crs='EPSG:27700')

out_dir = Path("data") / "raw_data"
out_dir.mkdir(parents=True, exist_ok=True)

pm10_out_path = out_dir / "PM10.parquet"
PM10_gdf.to_parquet(pm10_out_path, index=False)
print("Saved PM10 geoparquet to", pm10_out_path)

no2_out_path = out_dir / "NO2.parquet"
NO2_gdf.to_parquet(no2_out_path, index=False)
print("Saved NO2 geoparquet to", no2_out_path)

so2_out_path = out_dir / "SO2.parquet"
SO2_gdf.to_parquet(so2_out_path, index=False)
print("Saved SO2 geoparquet to", so2_out_path)

Saved PM10 geoparquet to data/raw_data/PM10.parquet
Saved NO2 geoparquet to data/raw_data/NO2.parquet
Saved SO2 geoparquet to data/raw_data/SO2.parquet


In [166]:
# Function to calculate area-weighted average of gridded point data for polygons

def area_weighted_average(
    points_gdf: gpd.GeoDataFrame,
    polygons_gdf: gpd.GeoDataFrame,
    value_col: str,
    grid_size: float = 1000,
    polygon_id_col: str = None
) -> gpd.GeoDataFrame:
    """
    Calculate area-weighted average of gridded point data for each polygon.
    
    Parameters
    ----------
    points_gdf : GeoDataFrame
        Point centroids of grid cells with values to aggregate
    polygons_gdf : GeoDataFrame
        Polygon boundaries to aggregate into
    value_col : str
        Column name in points_gdf containing values to average
    grid_size : float
        Size of grid cells in CRS units (default 1000m for 1km grid)
    polygon_id_col : str, optional
        Column to use as polygon identifier. If None, uses index.
    
    Returns
    -------
    GeoDataFrame
        Original polygons with new column '{value_col}_weighted_mean'
    """
    
    # Work with copies to avoid modifying originals
    points = points_gdf.copy()
    polygons = polygons_gdf.copy()
    
    # Ensure same CRS
    if points.crs != polygons.crs:
        points = points.to_crs(polygons.crs)
    
    # Create polygon ID if not specified
    if polygon_id_col is None:
        polygons['_poly_id'] = polygons.index
        polygon_id_col = '_poly_id'
    
    # Convert point centroids to grid squares
    half_grid = grid_size / 2
    points['geometry'] = points.geometry.buffer(half_grid, cap_style=3)
    
    # Keep only necessary columns for overlay
    points_subset = points[[value_col, 'geometry']].copy()
    polygons_subset = polygons[[polygon_id_col, 'geometry']].copy()
    
    # Intersect grids with boundaries
    intersected = gpd.overlay(points_subset, polygons_subset, how='intersection')
    
    # Calculate intersection areas
    intersected['_intersect_area'] = intersected.geometry.area
    
    # Calculate weighted values
    intersected['_weighted_value'] = intersected[value_col] * intersected['_intersect_area']
    
    # Aggregate by polygon
    agg = intersected.groupby(polygon_id_col).agg({
        '_weighted_value': 'sum',
        '_intersect_area': 'sum'
    }).reset_index()
    
    # Calculate weighted mean
    result_col = f'{value_col}_weighted_mean'
    agg[result_col] = agg['_weighted_value'] / agg['_intersect_area']
    
    # Merge back to original polygons
    result = polygons.merge(
        agg[[polygon_id_col, result_col]], 
        on=polygon_id_col, 
        how='left'
    )
    
    # Clean up temp columns
    if '_poly_id' in result.columns:
        result = result.drop(columns=['_poly_id'])
    
    return result



In [167]:
# Calculate area-weighted averages for each pollutant
# PM10
boundaries_with_pm10 = area_weighted_average(
    points_gdf=PM10_gdf,
    polygons_gdf=boundaries_all,
    value_col='pm102024g',
    grid_size=1000
)

# NO2
boundaries_with_no2 = area_weighted_average(
    points_gdf=NO2_gdf,
    polygons_gdf=boundaries_all,
    value_col='nox2024',
    grid_size=1000
)

# SO2
boundaries_with_so2 = area_weighted_average(
    points_gdf=SO2_gdf,
    polygons_gdf=boundaries_all,
    value_col='so22024',
    grid_size=1000
)

out_dir = Path("data") / "airquality"
out_dir.mkdir(parents=True, exist_ok=True)

pm10_out_path = out_dir / "pm10.parquet"
boundaries_with_pm10.to_parquet(pm10_out_path, index=False)
print("Saved boundaries_with_pm10 geoparquet to", pm10_out_path)

no2_out_path = out_dir / "no2.parquet"
boundaries_with_no2.to_parquet(no2_out_path, index=False)
print("Saved boundaries_with_no2 geoparquet to", no2_out_path)

so2_out_path = out_dir / "so2.parquet"
boundaries_with_so2.to_parquet(so2_out_path, index=False)
print("Saved boundaries_with_so2 geoparquet to", so2_out_path)

/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 467 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 467 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 467 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  resu

# Green / Blue Space
## Blue Space

The following code requires Osmium - https://osmcode.org/osmium-tool/

In [176]:
# Download Great Britain OSM PBF file
url = "https://download.geofabrik.de/europe/great-britain-latest.osm.pbf"
out_dir = Path("data") / "raw_data"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "great-britain-latest.osm.pbf"
urllib.request.urlretrieve(url, out_path)

(PosixPath('data/raw_data/great-britain-latest.osm.pbf'),
 <http.client.HTTPMessage at 0x51625e4b0>)

In [178]:
# Filter for water features
subprocess.run([
    "osmium", "tags-filter", "data/raw_data/great-britain-latest.osm.pbf",
    "nwr/natural=water", "w/waterway=*", "-o", "data/raw_data/gb-water.osm.pbf"
], check=True)

# Export to GeoJSON
subprocess.run([
    "osmium", "export", "data/raw_data/gb-water.osm.pbf",
    "-o", "data/raw_data/gb-water.geojson"
], check=True)

# Convert to Parquet
subprocess.run([
    "ogr2ogr", "-f", "Parquet", "data/raw_data/gb-water.parquet",
    "data/raw_data/gb-water.geojson"
], check=True)

# Filter for coastline
subprocess.run([
    "osmium", "tags-filter", "data/raw_data/great-britain-latest.osm.pbf",
    "w/natural=coastline", "-o", "data/raw_data/gb-coast.osm.pbf"
], check=True)

# Export to GeoJSON
subprocess.run([
    "osmium", "export", "data/raw_data/gb-coast.osm.pbf",
    "-o", "data/raw_data/gb-coast.geojson"
], check=True)

# Convert to Parquet
subprocess.run([
    "ogr2ogr", "-f", "Parquet", "data/raw_data/gb-coast.parquet",
    "data/raw_data/gb-coast.geojson"
], check=True)

# Delete the GeoJSON / PBF files after conversion
os.remove("data/raw_data/gb-water.geojson")
os.remove("data/raw_data/gb-coast.geojson")

os.remove("data/raw_data/gb-water.osm.pbf")
os.remove("data/raw_data/gb-coast.osm.pbf")

out_path.unlink() # remove the original pbf file

ERROR 1: PROJ: proj_create_from_database: /opt/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.
Warning 1: Field 'area' already exists. Renaming it as 'area2'
Warning 1: Field 'fixme' already exists. Renaming it as 'fixme2'
Warning 1: Field 'notes' already exists. Renaming it as 'notes2'
Warning 1: Field 'todo' already exists. Renaming it as 'todo2'
ERROR 1: PROJ: proj_create_from_database: /opt/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.
Warning 1: Field 'fixme' already exists. Renaming it as 'fixme2'


In [11]:
# Import the coastline and larger water bodies
gb_coast = gpd.read_parquet("data/raw_data/gb-coast.parquet")
gb_coast = gb_coast[gb_coast.geometry.type == 'LineString']
gb_water = gpd.read_parquet("data/raw_data/gb-water.parquet")
gb_water = gb_water[gb_water.geometry.type.isin(['Polygon', 'MultiPolygon'])]

In [26]:
# Convert to OSGB (British National Grid) and calculate areas
gb_water = gb_water.to_crs('EPSG:27700')

# Calculate areas for water polygons
gb_water['area_m2'] = gb_water.geometry.area
gb_water = gb_water[gb_water['area_m2'] >= 7500].copy()
gb_water = gb_water.to_crs(gb_coast.crs)

In [18]:
# Import OS Rivers Data

# Download OS Open Rivers data
rivers_url = "https://api.os.uk/downloads/v1/products/OpenRivers/downloads?area=GB&format=GeoPackage&redirect"

rivers_tmp_dir = tempfile.mkdtemp()
rivers_zip_path = os.path.join(rivers_tmp_dir, "oprvrs_gpkg_gb.zip")

response = requests.get(rivers_url, allow_redirects=True, timeout=120)
response.raise_for_status()

print(f"Downloaded {len(response.content) / 1024**2:.1f} MB")

with open(rivers_zip_path, 'wb') as f:
    f.write(response.content)

# Extract
with zipfile.ZipFile(rivers_zip_path, 'r') as zip_ref:
    zip_ref.extractall(rivers_tmp_dir)

rivers_gpkg_path = os.path.join(rivers_tmp_dir, "Data", "oprvrs_gb.gpkg")

# Read the watercourse_link layer which contains LineStrings (not the default hydro_node layer)
gb_rivers = gpd.read_file(rivers_gpkg_path, layer='watercourse_link')
print(f"Loaded {len(gb_rivers):,} river features")

# Verify we have LineStrings
print(f"Geometry types: {gb_rivers.geometry.type.unique()}")

Downloaded 49.9 MB
Loaded 192,738 river features
Geometry types: ['LineString']


In [20]:
# Filter gb_rivers to keep only LineString geometries
gb_rivers = gb_rivers.to_crs(gb_coast.crs)
gb_rivers

,id,flow_direction,length,fictitious,form,watercourse_name,watercourse_name_alternative,start_node,end_node,geometry
0,73E1A217-C5BF-41E3-AA17-F969F5181E04,in direction,18.0,0,tidalRiver,None,None,6B77D6D2-B9B2-4FBE-AEA3-8A801079B231,35806C2B-E01E-4CB8-9767-3D6F86302713,"LINESTRING (-0.83541 60.7855, -0.83519 60.78562)"
1,A44E2603-3307-4555-9604-3DAB8AA308C6,in direction,1009.0,0,inlandRiver,None,None,8FA84F8B-71CE-4626-8CEC-5964B94D34CC,D2571525-C2FD-4A2A-8236-7E938081B3E7,"LINESTRING (-0.83421 60.79485, -0.8334 60.7948..."
2,83332CB1-EC9C-486C-AC88-0BE480176548,in direction,1352.0,0,inlandRiver,Burn of Norwick,None,5EEC6ADE-DF7B-4B72-BE47-C78F810C39BF,C3EE6BFE-7B45-4251-951D-4F2A3BB4BBC3,"LINESTRING (-0.82643 60.80476, -0.82665 60.804..."
3,83C8021E-C7EF-4C2B-955A-3365D9DDF079,in direction,165.0,0,inlandRiver,None,None,5CF9C9E6-F0D3-4421-81AD-989455DD490C,5EEC6ADE-DF7B-4B72-BE47-C78F810C39BF,"LINESTRING (-0.82584 60.80594, -0.82555 60.805..."
4,58F9A511-A4BD-47F4-95C1-FE396F677E1B,in direction,27.0,0,inlandRiver,None,None,D2571525-C2FD-4A2A-8236-7E938081B3E7,1740F423-BAF7-4B61-AE72-743CC7BE3D60,"LINESTRING (-0.82285 60.78971, -0.82286 60.78947)"
...,...,...,...,...,...,...,...,...,...,...
192733,7EB00B29-3123-49FD-AD44-8340BB33F404,in direction,232.0,0,inlandRiver,None,None,04E1EF1B-618C-4BBF-A345-5C783A897E02,5EE99987-0230-4BBB-BDFC-70C96B139C28,"LINESTRING (-5.61168 50.14567, -5.6119 50.1455..."
192734,3355B2D8-576B-430D-BE7D-382B926F0FE7,in direction,732.0,0,inlandRiver,None,None,937C50B0-E3C8-4006-8C05-299E0D3B7334,1FEFFD80-26F0-4BB0-880A-63DE127AD449,"LINESTRING (-5.56626 50.16696, -5.56618 50.166..."
192735,EE3F229D-62CF-4877-B57F-ADE942804371,in direction,568.0,0,inlandRiver,None,None,03D09FEA-9748-456F-9B92-F0B657624E1C,56037488-0E8B-4A60-8934-7EE1339012EB,"LINESTRING (-5.65054 50.16165, -5.65119 50.162..."
192736,AC23A1FC-7C84-48AF-9E49-EDD08442AAE1,in direction,733.0,0,inlandRiver,None,None,2F32710C-3CF5-4EEF-B5F8-7D4CFAE7429B,3C77BFD9-D00F-40A5-9D5B-D916B99638CD,"LINESTRING (-5.60002 50.17741, -5.60035 50.177..."


In [27]:


# Convert LineStrings to points
from shapely import get_coordinates

coords = get_coordinates(gb_water.geometry)
gb_water_points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]),
    crs=gb_water.crs
)

coords = get_coordinates(gb_coast.geometry)
gb_coast_points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]),
    crs=gb_coast.crs
)

coords = get_coordinates(gb_rivers.geometry)
gb_rivers_points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]),
    crs=gb_rivers.crs
)

# Combine gb_water_points, gb_coast_points, and gb_rivers_points
gb_blue_space = gpd.GeoDataFrame(
    pd.concat([gb_water_points, gb_coast_points, gb_rivers_points], ignore_index=True),
    geometry="geometry",
    crs=gb_water_points.crs
)

# Save to parquet
blue_space_dir = Path("data") / "green_blue"
blue_space_dir.mkdir(parents=True, exist_ok=True)
blue_space_out_path = blue_space_dir / "bluespace.parquet"
gb_blue_space.to_parquet(blue_space_out_path, index=False)
print(f"Saved combined blue space points to {blue_space_out_path}")
print(f"Total points: {len(gb_blue_space):,}")

Saved combined blue space points to data/green_blue/bluespace.parquet
Total points: 8,574,806


# Greenspace
## Ambient Greenspace

The following code generates the ambient Greenspace indicator.

### Google Cloud Setup

There are a few steps to set up Google Cloud for use with Earth Engine and Cloud Storage.

1. **Install Google Cloud SDK**
For example - on macOS with MacPorts

```sudo port install google-cloud-sdk```

2. **Authenticate with Google Cloud**
The following command needs to be run in a terminal and will open a browser window for authentication

```bashgcloud auth login```

3. **Create a Google Cloud Project**

Go to [console.cloud.google.com](https://console.cloud.google.com/)
Create a new project or select an existing one
Note your project ID

4. **Enable Required APIs**

Enable the Earth Engine API in the project
Enable the Cloud Storage API in the project

5. **Register for Earth Engine**

Visit code.earthengine.google.com and register your Google account

6. **Create a Cloud Storage Bucket**

In Cloud Console, go to Cloud Storage and "Create bucket"
The following code expects a bucket named 'a-h-a-h'


### Create a UK Boundary

In [ ]:
boundaries_all_UK = gpd.read_parquet(Path("data") / "boundary" / "LSOA_DZ_SDZ_21_22.parquet")

zip_url = "https://www.nisra.gov.uk/files/nisra/publications/geography-sdz2021-esri-shapefile.zip"

# create a temporary directory and download the zip there
tmp_dir = tempfile.mkdtemp()
zip_path = os.path.join(tmp_dir, "sdz2021.zip")
urllib.request.urlretrieve(zip_url, zip_path)

# Extract and import Northern Ireland SDZ 2021 shapefile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(tmp_dir)

# Read the shapefile
NI_boundaries = gpd.read_file(os.path.join(tmp_dir, "SDZ2021.shp"))

NI_boundaries = NI_boundaries.rename(columns={"SDZ2021_cd": "LSOA_DZ_SDZ_21_22"})
NI_boundaries = NI_boundaries[["LSOA_DZ_SDZ_21_22", "geometry"]].copy()

NI_boundaries = NI_boundaries.to_crs(boundaries_all.crs)

boundaries_all_UK = gpd.GeoDataFrame(
    pd.concat([boundaries_all, NI_boundaries], ignore_index=True),
    geometry="geometry",
    crs=boundaries_all.crs
)

# --- Reproject to WGS84 (required for GEE) ---
boundaries_gee = boundaries_all_UK.to_crs('EPSG:4326')

# Export to shapefile
output_dir = Path('./data/raw_data/gee_upload')
output_dir.mkdir(exist_ok=True)

shp_path = output_dir / 'boundaries_uk.shp'
boundaries_gee.to_file(shp_path, driver='ESRI Shapefile')

print(f'Saved to {shp_path}')

/var/folders/cr/2v6lbkmd1s3_lyxghzghl0rm0000gn/T/ipykernel_12695/1023555476.py:36: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  boundaries_gee.to_file(shp_path, driver='ESRI Shapefile')
/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'LSOA_DZ_SDZ_21_22' to 'LSOA_DZ_SD'
  ogr_write(


Saved to data/raw_data/gee_upload/boundaries_uk.shp


### Setup

In [ ]:
# Initialize Earth Engine with a specific project
ee.Authenticate()
ee.Initialize(project='ndvi-inspire')

# Set the project
result = subprocess.run(['earthengine', 'set_project', 'ndvi-inspire'], capture_output=True, text=True)

#### Upload Shapefile to GCS

In [ ]:
# Upload all shapefile components to GCS ---
bucket = 'a-h-a-h'
gcs_folder = 'gee_uploads'

shp_extensions = ['.shp', '.shx', '.dbf', '.prj', '.cpg']

for ext in shp_extensions:
    local_file = shp_path.with_suffix(ext)
    if local_file.exists():
        gcs_path = f'gs://{bucket}/{gcs_folder}/{local_file.name}'
        print(f'Uploading {local_file.name}...')
        subprocess.run(['gsutil', 'cp', str(local_file), gcs_path], check=True)

print('All files uploaded to GCS')

# --- Step 4: Ingest into Earth Engine ---
asset_id = 'projects/ndvi-inspire/assets/boundaries_uk'
gcs_shp = f'gs://{bucket}/{gcs_folder}/boundaries_uk.shp'

cmd = [
    'earthengine',
    'upload',
    'table',
    f'--asset_id={asset_id}',
    gcs_shp
]

print(f'Running: {" ".join(cmd)}')
result = subprocess.run(cmd, capture_output=True, text=True)

print('stdout:', result.stdout)
print('stderr:', result.stderr)

Reprojected from {"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "ProjectedCRS", "name": "OSGB36 / British National Grid", "base_crs": {"name": "OSGB36", "datum": {"type": "GeodeticReferenceFrame", "name": "Ordnance Survey of Great Britain 1936", "ellipsoid": {"name": "Airy 1830", "semi_major_axis": 6377563.396, "inverse_flattening": 299.3249646}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "id": {"authority": "EPSG", "code": 4277}}, "conversion": {"name": "British National Grid", "method": {"name": "Transverse Mercator", "id": {"authority": "EPSG", "code": 9807}}, "parameters": [{"name": "Latitude of natural origin", "value": 49, "unit": "degree", "id": {"authority": "EPSG", "code": 8801}}, {"name": "Longitude of natural origin", "value": -2, "unit": "degree"

/var/folders/cr/2v6lbkmd1s3_lyxghzghl0rm0000gn/T/ipykernel_12695/2701278731.py:15: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  boundaries_gee.to_file(shp_path, driver='ESRI Shapefile')
/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/pyogrio/raw.py:733: RuntimeWarning: Normalized/laundered field name: 'LSOA_DZ_SDZ_21_22' to 'LSOA_DZ_SD'
  ogr_write(


Saved to gee_upload/boundaries_uk.shp
Uploading boundaries_uk.shp...


Copying file://gee_upload/boundaries_uk.shp [Content-Type=application/octet-stream]...
- [1 files][112.2 MiB/112.2 MiB]                                                
Operation completed over 1 objects/112.2 MiB.                                    


Uploading boundaries_uk.shx...


...

Uploading boundaries_uk.prj...


Copying file://gee_upload/boundaries_uk.prj [Content-Type=application/octet-stream]...
/ [1 files][  145.0 B/  145.0 B]                                                
Operation completed over 1 objects/145.0 B.                                      


Uploading boundaries_uk.cpg...


Copying file://gee_upload/boundaries_uk.cpg [Content-Type=application/octet-stream]...
/ [1 files][    5.0 B/    5.0 B]                                                
Operation completed over 1 objects/5.0 B.                                        


All files uploaded to GCS
Running: earthengine upload table --asset_id=projects/ndvi-inspire/assets/boundaries_uk gs://a-h-a-h/gee_uploads/boundaries_uk.shp
stdout: Started upload task with ID: A3WF3I3BUPXU5YKLUU7JEK5A

stderr: 


### Extract Greenspace within LSOAs

In [ ]:
# Cloud masking for Sentinel-2
def mask_s2_clouds(image):
    """Mask clouds using SCL band, but keep original for pixel counting"""
    scl = image.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    
    total_pixels = ee.Image.constant(1).rename('total_pixels')
    valid_pixels = ee.Image.constant(1).updateMask(mask).rename('valid_pixels')
    
    return image.updateMask(mask).addBands([total_pixels, valid_pixels])


def add_indices(image):
    """Calculate NDVI, EVI, and Fractional Vegetation Cover"""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    nir = image.select('B8').divide(10000)
    red = image.select('B4').divide(10000)
    blue = image.select('B2').divide(10000)
    
    evi = nir.subtract(red).multiply(2.5).divide(
        nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)
    ).rename('EVI')
    
    # Fractional Vegetation Cover
    ndvi_soil = 0.2
    ndvi_veg = 0.86
    fvc = ndvi.subtract(ndvi_soil).divide(ndvi_veg - ndvi_soil).pow(2).clamp(0, 1).rename('FVC')
    
    return image.addBands([ndvi, evi, fvc])


# Main processing
start_date = '2025-04-01'
end_date = '2025-10-31'

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterDate(start_date, end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      .map(mask_s2_clouds)
      .map(add_indices))

# Create composite
composite = s2.select(['NDVI', 'EVI', 'FVC']).median()

# Pixel counts - sum across collection
pixel_counts = s2.select(['total_pixels', 'valid_pixels']).sum()

# Build combined reducer for all stats
reducer = (
    ee.Reducer.mean()
    .combine(ee.Reducer.median(), sharedInputs=True)
    .combine(ee.Reducer.stdDev(), sharedInputs=True)
    .combine(ee.Reducer.max(), sharedInputs=True)
    .combine(ee.Reducer.min(), sharedInputs=True)
    .combine(ee.Reducer.count(), sharedInputs=True)
)

# Load the boundaries
boundaries = ee.FeatureCollection('projects/ndvi-inspire/assets/boundaries_uk')

print(f'Loaded {boundaries.size().getInfo()} features')

# Extract statistics
veg_stats = composite.reduceRegions(
    collection=boundaries,
    reducer=reducer,
    scale=10
)

pixel_stats = pixel_counts.reduceRegions(
    collection=boundaries,
    reducer=ee.Reducer.sum(),
    scale=10
)

# Export to your GCS bucket
task1 = ee.batch.Export.table.toCloudStorage(
    collection=veg_stats,
    description='uk_veg_stats',
    bucket='a-h-a-h',
    fileNamePrefix='gee_exports/uk_veg_stats',
    fileFormat='CSV'
)

task2 = ee.batch.Export.table.toCloudStorage(
    collection=pixel_stats,
    description='uk_pixel_counts',
    bucket='a-h-a-h',
    fileNamePrefix='gee_exports/uk_pixel_counts',
    fileFormat='CSV'
)

task1.start()
task2.start()

print('Export tasks started')

# Create the output directory for GEE exports if it doesn't exist
output_dir = Path('data/raw_data/gee_exports')
output_dir.mkdir(parents=True, exist_ok=True)

# Download the vegetation statistics CSV from Google Cloud Storage
subprocess.run(['gsutil', 'cp', 'gs://a-h-a-h/gee_exports/uk_veg_stats.csv', str(output_dir)], check=True)
# Download the pixel counts CSV from Google Cloud Storage
subprocess.run(['gsutil', 'cp', 'gs://a-h-a-h/gee_exports/uk_pixel_counts.csv', str(output_dir)], check=True)

# Read the downloaded CSV files into pandas DataFrames
veg_stats = pd.read_csv(output_dir / 'uk_veg_stats.csv')
pixel_counts = pd.read_csv(output_dir / 'uk_pixel_counts.csv')

# Clean and merge
veg_stats = veg_stats[['LSOA_DZ_SD', 'EVI_count', 'EVI_max', 'EVI_mean', 'EVI_median', 'EVI_min', 'EVI_stdDev', 'FVC_count', 'FVC_max', 'FVC_mean', 'FVC_median', 'FVC_min', 'FVC_stdDev', 'NDVI_count', 'NDVI_max', 'NDVI_mean', 'NDVI_median', 'NDVI_min', 'NDVI_stdDev']]
pixel_counts = pixel_counts.drop(columns=['system:index', '.geo'])
veg_stats = veg_stats.merge(pixel_counts, on='LSOA_DZ_SD', how='left')

# Merge veg_stats into boundaries_all_UK by matching LSOA IDs
boundaries_all_UK = boundaries_all_UK.merge(
    veg_stats,
    left_on="LSOA_DZ_SDZ_21_22",
    right_on="LSOA_DZ_SD",
    how="left"
).drop(columns=["LSOA_DZ_SD"], errors="ignore")

out_dir = Path("data") / "raw_data" / "gee_exports"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "Vegetation_Indicies_2025.parquet"
boundaries_all_UK.to_parquet(out_path, index=False)
print("Saved geoparquet to", out_path)


### Monitoring the Task
The following code can be run to monitor the status of the tasks.

In [116]:
# Monitor task progress
import subprocess
result = subprocess.run(['earthengine', 'task', 'list'], capture_output=True, text=True)
print(result.stdout)

PTAO676EZN5NHJ43QYTHUNKQ  Export.table  uk_pixel_counts                             SUCCEEDED  ---
6CY63UC6N3BVDRI2MATOKVGM  Export.table  uk_veg_stats                                SUCCEEDED  ---
2TKRAP2X4CRSS4ZXCZVAJJZT  Export.table  uk_pixel_counts                             SUCCEEDED  ---
NP4ICG7KIZYPCOTNRT7SIKSZ  Export.table  uk_veg_stats                                SUCCEEDED  ---
A3WF3I3BUPXU5YKLUU7JEK5A  Upload        Ingest table: "projects/ndvi-inspire/ass..  SUCCEEDED  ---



### Create AHAH Greenspace Parquet File

In [34]:
ndvi_path = Path("data/raw_data/gee_exports/Vegetation_Indicies_2025.parquet")
df_ndvi_median = gpd.read_parquet(ndvi_path)
df_ndvi_median = df_ndvi_median.rename(columns={"LSOA_DZ_SD": "LSOA_DZ_SDZ_21_22"})
df_ndvi_median = df_ndvi_median[['LSOA_DZ_SDZ_21_22', 'geometry', 'NDVI_median']].copy()
out_dir = Path("data") / "green_blue"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "greenspace.parquet"
df_ndvi_median.to_parquet(out_path, index=False)
print("Saved greenspace geoparquet to", out_path)

Saved greenspace geoparquet to data/green_blue/greenspace.parquet


## Active Greenspace
The following code generates the active Greenspace indicator using Ordnace Survey Open Greenspace data.

In [ ]:
import requests
import tempfile
import zipfile
import os
import geopandas as gpd

tmp_dir = tempfile.mkdtemp()
zip_path = os.path.join(tmp_dir, "opgrsp_gpkg_gb.zip")

url = "https://api.os.uk/downloads/v1/products/OpenGreenspace/downloads?area=GB&format=GeoPackage&redirect"

response = requests.get(url, allow_redirects=True, timeout=120)
response.raise_for_status()

print(f"Downloaded {len(response.content) / 1024**2:.1f} MB")

with open(zip_path, 'wb') as f:
    f.write(response.content)

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(tmp_dir)

gpkg_path = os.path.join(tmp_dir, "Data", "opgrsp_gb.gpkg")

active_greenspace_polygons = gpd.read_file(gpkg_path, layer='greenspace_site')


Downloaded 54.6 MB


/Users/alex/github/access_to_cash_2025/.conda/lib/python3.12/site-packages/pyogrio/geopandas.py:382: UserWarning: More than one layer found in 'opgrsp_gb.gpkg': 'access_point' (default), 'greenspace_site'. Specify layer parameter to avoid this warning.
  result = read_func(


In [254]:
active_greenspace_polygons = gpd.read_file(gpkg_path, layer='greenspace_site')

# Convert LineStrings to points
from shapely import get_coordinates

coords = get_coordinates(active_greenspace_polygons.geometry)

gb_green_points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]),
    crs=active_greenspace_polygons.crs
)

gb_green_points = gb_green_points.to_crs('EPSG:4326')

# Save to parquet
green_space_dir = Path("data") / "green_blue"
green_space_dir.mkdir(parents=True, exist_ok=True)
green_space_out_path = green_space_dir / "greenspace_active.parquet"
gb_green_points.to_parquet(green_space_out_path, index=False)
print(f"Saved combined blue space points to {green_space_out_path}")
print(f"Total points: {len(gb_green_points):,}")

Saved combined blue space points to data/green_blue/greenspace_active.parquet
Total points: 2,582,449
